Kaggle Challenge - Fraud Detection - BGU 2019

(1) Load Data

In [14]:
from utils import *
dataset = pd.read_csv('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/partial_labels.csv')
x,segments = dataset.axes
array = dataset.values
results_df = pd.DataFrame(columns=["id","label"])

(2) Pre-Processing

In [15]:
all_instructions = np.empty((150, 100))

for testUser in range(10,40):
    print("testUser = {}".format(testUser))
    z = np.reshape(array[testUser,1:],-1)
    Y_train = z[0:50]
    Y_train = Y_train.astype(int)
    #read instructions
    fileName = array[testUser,0]
    path_read = "../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/User"+str(testUser)
    instructions = pd.read_csv(path_read, sep=" ", header=None)
    arr = np.reshape(instructions.values,(150,100))
    print("arr = {}".format(arr))
    print("arr.shape = {}".format(arr.shape))
    all_instructions = np.concatenate((all_instructions, arr),axis=0)

testUser = 10
arr = [['cpp' 'sh' 'xrdb' ... 'netscape' 'netscape' 'netscape']
 ['netscape' 'netscape' 'rlogin' ... 'csh' 'hostname' 'stty']
 ['tset' 'launchef' 'sh' ... 'sed' 'FIFO' 'rm']
 ...
 ['getopt' 'true' 'true' ... 'sh' 'csh' 'kill']
 ['userenv' 'wait4wm' 'xhost' ... 'launchef' 'sh' 'launchef']
 ['hostname' 'cat' 'mail' ... 'sendmail' 'sendmail' 'sendmail']]
arr.shape = (150, 100)
testUser = 11
arr = [['touch' 'touch' 'cat' ... 'cat' 'tty' 'cat']
 ['hostname' 'tty' 'arch' ... 'MediaMai' 'launchef' 'launchef']
 ['sh' 'gettxt' 'hostname' ... 'more' 'ls' 'ls']
 ...
 ['emacs-20' 'ksh' 'uname' ... 'emacs-20' 'uname' 'nawk']
 ['cpp' 'cc1' 'as' ... 'ln' 'uname' 'nawk']
 ['cpp' 'cc1' 'as' ... 'uname' 'nawk' 'cpp']]
arr.shape = (150, 100)
testUser = 12
arr = [['cpp' 'sh' 'xrdb' ... 'launchef' 'sh' 'launchef']
 ['hostname' '[' 'cat' ... 'getopt' 'true' 'grep']
 ['date' 'lp' 'find' ... 'FIFO' 'cat' 'date']
 ...
 ['hostname' 'tset' 'hostname' ... 'netscape' 'netscape' 'netscape']
 ['mp' 'ca

(3) Feature Engineering

In [16]:
uniqe=set()
index=[]
z=0
x_train=[]

index=0
wordsCount={}
countbackwrd= set()
len_countbackwrd=[]
word_lenght=[]
count = 0
mean_len=0

count_vec_usr=[]
vectorizer = TfidfVectorizer(min_df=0.4,ngram_range=(1,2))
count_vec_all=[]
cout_len=0

for i in range(150):
    Str = ' '.join((all_instructions[i,:]))
    if(i<50):
        count_vec_usr.append(' '.join(all_instructions[i,:]))
        count_vec_all.append(' '.join(all_instructions[i,:]))
    else:
        count_vec_all.append(' '.join(all_instructions[i,:]))
        
vectorizer.fit_transform(count_vec_all)
X_all = vectorizer.transform(count_vec_all)
X_all=X_all.toarray()
X_usr = vectorizer.transform(count_vec_usr)
X_usr=X_usr.toarray()

for i in range(50):
    for j in range(100):
        uniqe.add(all_instructions[i][j])
        for z in range(j,j+5):
            if(j%5==0 or j==0):
                countbackwrd.add(all_instructions[i][z])
        mean_len=len(countbackwrd)+mean_len
        cout_len=len(all_instructions[i][j])+cout_len
        countbackwrd.clear()
    x_train.append([random.randint(0, 2)])
    mean_len=0
    cout_len=0
    
mean_len=0
count=0
strange=[]
uniqcheck=set()

for i in range(50,150):
    for j in range(100):
        uniqcheck.add(all_instructions[i][j])
        if (all_instructions[i][j] not in uniqe):
            count = count + 1
        for z in range(j,j+5):
            if(j%5==0 or j==0):
                countbackwrd.add(all_instructions[i][z])
        mean_len=len(countbackwrd)+mean_len
        cout_len=len(all_instructions[i][j])+cout_len
        countbackwrd.clear()
    strange.append([count])
    uniqcheck.clear()
    count = 0
    mean_len=0
    cout_len=0

TypeError: sequence item 0: expected str instance, float found

(4) Model Fit & Predict

In [ ]:
#train module
#x_test=x_train+strange
x_test=np.concatenate((x_train,strange))
#x_test=np.c_[X_all,x_test]
#x_train=np.c_[X_usr,x_train]
clf= LocalOutlierFactor(n_neighbors=25,contamination=0.1, novelty=True)
#clf = IsolationForest(behaviour='new', max_samples=20, contamination=0.5)
#clf=EllipticEnvelope(support_fraction=1.,contamination=0.1)
clf.fit(x_train)
t=clf.predict(x_test)
t[t == 1] = 0
t[t == -1] = 1
plt.show()
print(X_usr)

(5) Push Results to Kaggle Challenge

In [ ]:
curr_user_results = t[50:150]
curr_user_results_df = pd.DataFrame(columns=["id","label"])

start_rows = 5000
till_rows = 5100

for i in range(curr_user_results.size):
    curr_user_results_df.loc[i] = ["User{}_{}-{}".format(testUser,start_rows,till_rows),curr_user_results[i]]
    start_rows = start_rows + 100
    till_rows = till_rows + 100
results_df = results_df.append(curr_user_results_df)

fname = "out/submission_results_{}.csv".format(date.today())
results_df.to_csv(fname, index=False)
#os.system('kaggle competitions submit -c detectingcyberattackscallenge -f "{0}" -m "{1}"'.format(fname, "Submitting results to kaggle"))